Adaptando dataset

In [1]:
import json

ANN_TRAIN = "/home/messyas/ml/data/tccdata/annotations/4_class_splits/annotations_train.json"
ANN_VAL   = "/home/messyas/ml/data/tccdata/annotations/4_class_splits/annotations_val.json"

def get_class_names(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return [cat['name'] for cat in data.get('categories', [])]

train_names = get_class_names(ANN_TRAIN)
val_names   = get_class_names(ANN_VAL)

all_names = []
for name in train_names + val_names:
    if name not in all_names:
        all_names.append(name)

print("Classes encontradas:")
for idx, cls in enumerate(all_names):
    print(f"{idx}: {cls}")


Classes encontradas:
0: Plastic
1: Pile
2: Face mask
3: Trash bin


In [1]:
import os
import json
import shutil
import yaml
from PIL import Image


DATA_ROOT  = "/home/messyas/ml/data/tccdata"
IMG_DIR    = os.path.join(DATA_ROOT, "images")
ANN_DIR    = os.path.join(DATA_ROOT, "annotations/4_class_splits")
ANN_TRAIN  = os.path.join(ANN_DIR, "annotations_train.json")
ANN_VAL    = os.path.join(ANN_DIR, "annotations_val.json")
ANN_TEST   = os.path.join(ANN_DIR, "annotations_test.json")
YOLO_ROOT  = os.path.join(DATA_ROOT, "yolo_dataset")

for sub in ["images/train", "images/val", "images/test",
            "labels/train", "labels/val", "labels/test"]:
    os.makedirs(os.path.join(YOLO_ROOT, sub), exist_ok=True)

def normalize_and_filter(bbox, img_w, img_h):
    x, y, w, h = bbox
    x_c = (x + w/2) / img_w
    y_c = (y + h/2) / img_h
    w_n = w / img_w
    h_n = h / img_h
    if not (0 < x_c < 1 and 0 < y_c < 1 and 0 < w_n <= 1 and 0 < h_n <= 1):
        return None
    return x_c, y_c, w_n, h_n

def convert_split(json_path, split):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    imgs = {img['id']: img for img in data['images']}
    anns = data['annotations']
    cats = sorted(data['categories'], key=lambda c: c['id'])
    id2idx = {c['id']: i for i, c in enumerate(cats)}

    total_ann = 0
    invalid = 0
    duplicates = 0

    for img_id, img in imgs.items():
        filename = img.get('file_name', os.path.basename(img.get('path', '')))
        src = os.path.join(IMG_DIR, filename)
        dst_img = os.path.join(YOLO_ROOT, f"images/{split}", filename)
        shutil.copy(src, dst_img)
        w = img.get('width') or Image.open(src).size[0]
        h = img.get('height') or Image.open(src).size[1]

        lines = []
        for ann in [a for a in anns if a['image_id'] == img_id]:
            total_ann += 1
            norm = normalize_and_filter(ann['bbox'], w, h)
            if norm is None:
                invalid += 1
                continue
            cls = id2idx[ann['category_id']]
            lines.append(f"{cls} {norm[0]:.6f} {norm[1]:.6f} {norm[2]:.6f} {norm[3]:.6f}")
        unique = list(dict.fromkeys(lines))
        duplicates += len(lines) - len(unique)
        txt_path = os.path.join(YOLO_ROOT, f"labels/{split}", os.path.splitext(filename)[0] + ".txt")
        with open(txt_path, 'w') as f:
            f.write("\n".join(unique))

    print(f"{split}: {len(imgs)} imagens, {total_ann} caixas originais, {invalid} inválidas removidas, {duplicates} duplicadas removidas")
    return cats

cats = convert_split(ANN_TRAIN, 'train')
cats = convert_split(ANN_VAL,   'val')
cats = convert_split(ANN_TEST,  'test')

names = [c['name'] for c in cats]
cfg = {
    'path': YOLO_ROOT,
    'train': 'images/train',
    'val':   'images/val',
    'test':  'images/test',
    'nc':    len(names),
    'names': names
}
with open(os.path.join(YOLO_ROOT, 'data.yaml'), 'w') as f:
    yaml.dump(cfg, f, sort_keys=False)

print(f"Conversão completa em: {YOLO_ROOT}")
print(f"data.yaml gerado em: {os.path.join(YOLO_ROOT, 'data.yaml')}")

KeyboardInterrupt: 

Treinamento do modelo Yolov8x

In [2]:
from ultralytics import YOLO

model = YOLO("yolov8x.pt")
model.train(
    data      = "/home/messyas/ml/data/tccdata/yolo_dataset/data.yaml",
    epochs    = 100,
    imgsz     = 640,
    batch     = 4,
    device    = 0,
    augment   = True,
    patience  = 10
)

ERROR ❌ Error reading from /home/messyas/.config/Ultralytics/settings.json: "No Ultralytics setting 'openvino_msg'. \nView Ultralytics Settings with 'yolo settings' or at '/home/messyas/.config/Ultralytics/settings.json'\nUpdate Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings."
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/messyas/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
New https://pypi.org/project/ultralytics/8.3.161 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.129 🚀 Python-3.9.21 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 7965MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=4

KeyboardInterrupt: 